In [ ]:
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from IPython.display import clear_output
import rasterio as rio

In [ ]:
# You need to have a Planet API key in a file called "Planet API key.txt"
# in the current working directory
api_key_path = Path.cwd() / "Planet API key.txt"
api_key = api_key_path.read_text().strip()

In [3]:
download_dir = Path.cwd() / "dataset"
download_dir.mkdir(exist_ok=True)

In [5]:
scenes_dir = download_dir / "Scenes"
scenes_dir.mkdir(exist_ok=True)
udm2_dir = download_dir / "UDM2"
udm2_dir.mkdir(exist_ok=True)

In [ ]:
item_type = "PSScene"
asset_types = ["ortho_analytic_8b_sr", "ortho_analytic_8b_xml", "ortho_udm2"]

In [ ]:
val_items = [
    "20231017_074706_29_24cc",
    "20231020_040648_68_24ba",
    "20231027_104734_71_24ab",
    "20231028_112441_43_247d",
    "20231026_105053_18_247d",
    "20231023_100346_43_24bf",
    "20231028_100622_62_2440",
    "20231021_172556_48_24ab",
    "20231007_120440_68_24c0",
    "20231028_122611_96_2475",
    "20231006_134506_55_24b0",
    "20231028_122611_96_2475",
    "20231006_134506_55_24b0",
    "20230923_180857_76_2483",
    "20231012_170140_16_2495",
    "20231018_003511_81_24b4",
    "20231002_235222_22_24b9",
    "20230821_172926_55_24c4",
    "20230814_134236_48_242e",
    "20231010_020137_28_2424",
]
len(val_items)

In [ ]:
def download_file(url):
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        content_disposition = response.headers.get("content-disposition")
        if content_disposition:
            filename = content_disposition.split("filename=")[1].strip('"')
            file_path = download_dir / filename
            if file_path.exists():
                print(f"File {filename} already exists.")
                return

            with open(file_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
            print(f"File {filename} downloaded successfully.")
        else:
            print("Could not find a filename in the Content-Disposition header.")
    else:
        print("Failed to download the file.")

In [ ]:
all_item_dicts = []
for item in val_items:
    item_dicts = []
    item_url = (
        f"https://api.planet.com/data/v1/item-types/{item_type}/items/{item}/assets"
    )

    result = requests.get(item_url, auth=HTTPBasicAuth(api_key, ""))
    for asset_type in asset_types:
        links = result.json()[asset_type]["_links"]

        self_link = links["_self"]
        activation_link = links["activate"]
        item_links = {"self_link": self_link, "activation_link": activation_link}
        activate_result = requests.get(activation_link, auth=HTTPBasicAuth(api_key, ""))
        item_dicts.append(item_links)

    all_item_dicts.append(item_dicts)

In [ ]:
while True:
    all_prod_status = []

    for item_dicts in all_item_dicts:
        for item_dict in item_dicts:
            activation_status_result = requests.get(
                item_dict["self_link"], auth=HTTPBasicAuth(api_key, "")
            )
            product_info = activation_status_result.json()
            all_prod_status.append(product_info)

    clear_output(wait=True)
    results_df = pd.DataFrame(all_prod_status)
    print(results_df["status"].value_counts())
    if "activating" not in results_df["status"].to_list():
        download_links = pd.DataFrame(all_prod_status)["location"].to_list()
        break

In [ ]:
dl_threads = 3
with ThreadPoolExecutor(max_workers=dl_threads) as executor:
    executor.map(download_file, download_links)

In [ ]:
# move 8b.tif files into scenes dir
for scene_file in scenes_dir.glob("*8b.tif"):
    scene_file.rename(scenes_dir / scene_file.name)

In [ ]:
udm2_files = list(download_dir.glob("*udm2.tif"))
len(list(udm2_files))

In [ ]:
# flatten the UDM2 masks to a single band
flat_masks = []
for udm2_file in udm2_files:
    filename = udm2_file.name
    export_path = udm2_dir / f"{filename.replace('.tif', '_flat_mask.tif')}"
    src = rio.open(udm2_file)
    profile = src.profile
    raw_mask = src.read([1, 2, 3, 4, 5, 6])
    mask_max = raw_mask.argmax(axis=0, keepdims=True)
    profile.update(count=1, dtype=rio.uint8, compress="lzw")
    with rio.open(export_path, "w", **profile) as dst:
        dst.write(mask_max + 1)